# Find highly correlated features with winning the oscar and predict if movie will win the oscare or not by using different classifiers

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

In [2]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV, ElasticNetCV
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import confusion_matrix

from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from sklearn import svm 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

In [3]:
df = pd.read_csv('../../../Files/FinalFinalFinalClean.csv')

In [4]:
df.shape

(6698, 65)

In [5]:
df.columns

Index(['fid', 'fname', 'fruntime', 'fimdb_rating', 'fimdb_votes', 'fimd',
       'fdirector', 'fwriter', 'fassistant_directors', 'fcinematographers',
       'fcomposers', 'feditors', 'fproducer', 'fenglish', 'fmulti_lang', 'fus',
       'fmulti_country', 'fmystery', 'fmusic', 'faction', 'fsport', 'fmusical',
       'ffamily', 'fcomedy', 'fhorror', 'ffantasy', 'fdrama', 'fsci-fi',
       'fthriller', 'fwestern', 'fadventure', 'fhistory', 'fwar', 'fcrime',
       'fromance', 'fbiography', 'fanimation', 'actW', 'nominated', 'Won',
       'month', 'year', 'certificate_G', 'certificate_PG', 'certificate_PG-13',
       'certificate_R', 'certificate_Unrated', 'fproduction_BBC Films',
       'fproduction_Columbia Pictures', 'fproduction_Dimension Films',
       'fproduction_DreamWorks', 'fproduction_Fox Searchlight Pictures',
       'fproduction_Lionsgate', 'fproduction_Millennium Films',
       'fproduction_New Line Cinema', 'fproduction_Paramount Pictures',
       'fproduction_Screen Gems', 

In [6]:
df['fcumlative_y'].

SyntaxError: invalid syntax (<ipython-input-6-b3af38f3818e>, line 1)

### Sperate 2019 Movies

In [7]:
df2019 = df[df['year']== 2019]
df = df[df['year']< 2019]

# Classification

## Winning

## Find top correlated columns

In [8]:
corr = df.corr()

In [9]:
corrGross = pd.DataFrame(corr[abs(corr['Won'])> 0.1]['Won']).sort_values('Won')
grossFeatures = list(corrGross.index)

In [10]:
corrGross

,Won
fcomposers,0.105615
fdirector,0.113696
fbudget_y,0.115981
fimd,0.119748
fruntime,0.125207
fimdb_rating,0.206593
fcumlative_y,0.255712
fimdb_votes,0.396107
nominated,0.716875
Won,1.000000


In [11]:
y = df['Won']
X= df.drop(['fid', 'fname','Won','nominated'], axis=1)

In [12]:

#XC = X.columns
mms = MinMaxScaler()
#X.columns = XC 

X = pd.DataFrame(mms.fit_transform(X), columns = X.columns)

In [12]:

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier

random_state = 42
 
model_names = ['LogisticRegression', 'DecisionTreeClassifier', 'SVC', 
              'RandomForestClassifier', 'XGBClassifier', 'ExtraTreesClassifier'
              , 'GradientBoostingClassifier','AdaBoostClassifier','KNN']




LR = GridSearchCV(LogisticRegression(random_state=random_state),{"C":np.logspace(-3,3,7), "penalty":["l1","l2"]},cv=10)
DT = GridSearchCV(DecisionTreeClassifier(random_state=random_state),{'min_samples_split' : range(10,500,20),'max_depth': range(1,20,2)},cv=10)
SV =  GridSearchCV(SVC(random_state=random_state),[{'C': [5, 100, 1000], 'kernel': ['linear']},
  {'C': [5, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}, ],cv=10)
RF = GridSearchCV(RandomForestClassifier(random_state=42),{'n_estimators': [200, 700],'max_features': ['auto', 'sqrt', 'log2']},cv=10 )
XT = GridSearchCV(ExtraTreesClassifier(random_state=random_state),{'n_estimators': [16, 32] },cv=10)

#AB = GridSearchCV(AdaBoostClassifier(random_state=random_state), {'n_estimators': (1, 2),
#                  'base_estimator__max_depth': (1, 2),
#                  'algorithm': ('SAMME', 'SAMME.R')}, cv =10)

AB = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1),
    n_estimators=200)
KNN = GridSearchCV(KNeighborsClassifier(), {'n_neighbors':[3,5,7]}, cv =10)

#GB = GridSearchCV(GradientBoostingClassifier(random_state=random_state),{'learning_rate': [0.1, 0.05, 0.02],
#              'max_depth': [4, 6, 3],
#              'min_samples_leaf': [20, 50,100],
#              'max_features': [1.0, 0.5, 0.2] 
#              }, cv =10)


GB = GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=10)

models = [ ('LogisticRegression',LR),
           ('DecisionTreeClassifier',DT),
           ('SVC',SV),
          ('RandomForestClassifier',RF),
          ('ExtraTreesClassifier', XT),
          ('AdaBoostClassifier',AB),
          ('KNN',KNN),
          ('GradientBoostingClassifier',GB)
         ]


In [13]:
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)


In [ ]:
model_accuracy = []
for k,model in models:
    print (k , ':')
    model.fit(X, y)
    accuracy = cross_val_score(model, X, y, cv=5).mean()
    predicted = model.predict(X_test)
    print("precition: ",f1_score(y_test==1, predicted==1))
    print("Recall: ", recall_score(y_test, predicted))
    print(confusion_matrix(predicted, y_test))
    model_accuracy.append(accuracy)
    print(accuracy)
    print('\n')

LogisticRegression :
precition:  0.275
Recall:  0.2
[[2123   44]
 [  14   11]]
0.9768438101022371


DecisionTreeClassifier :
precition:  0.24324324324324323
Recall:  0.16363636363636364
[[2127   46]
 [  10    9]]
0.9766165603806053


SVC :


In [14]:


log_modal = LogisticRegression().fit(X_train,y_train)
cross_val_score(log_modal,X_train,y_train,cv= 10).mean()

C:\Users\baso0\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\baso0\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\baso0\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\baso0\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\baso0\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linea

0.9775188865996041

In [15]:
predicted = log_modal.predict(X_test)

In [16]:
print("precition: ",f1_score(y_test==1, predicted==1))
print("Recall: ", recall_score(y_test, predicted))

precition:  0.03389830508474576
Recall:  0.01818181818181818


In [17]:
from sklearn.metrics import confusion_matrix


confusion_matrix(predicted, y_test)

array([[2134,   54],
       [   3,    1]], dtype=int64)

In [15]:
cmap=sns.diverging_palette(5, 250, as_cmap=True)

def magnify():
    
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]
corrGross = corr[['fGross_y']].sort_values('fGross_y')
corr.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
   .set_table_styles(magnify())

In [16]:
cmap=sns.diverging_palette(5, 250, as_cmap=True)
corr = df.corr()
def magnify():
    
    return [dict(selector="th",
                 props=[("font-size", "7pt")]),
            dict(selector="td",
                 props=[('padding', "0em 0em")]),
            dict(selector="th:hover",
                 props=[("font-size", "12pt")]),
            dict(selector="tr:hover td:hover",
                 props=[('max-width', '200px'),
                        ('font-size', '12pt')])
]
corrGross = corr[['Won']].sort_values('Won')
corrGross.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
   .set_table_styles(magnify())

,Won
certificate_Unrated,-0.065
fproduction_other,-0.06
fcomedy,-0.055
fhorror,-0.053
faction,-0.029
fcrime,-0.027
fthriller,-0.02
fGross_y,-0.018
fsport,-0.016
fproduction_Lionsgate,-0.015


In [22]:
corrcumlative = pd.DataFrame(corr[abs(corr['fcumlative'])> 0.1]['fcumlative']).sort_values('fcumlative')
corrcumlative

,fcumlative
fproduction_other,-0.303618
fdrama,-0.225838
certificate_R,-0.150095
certificate_Unrated,-0.138803
fimd,0.106525
fcinematographers,0.110647
fmulti_lang,0.123630
fproduction_Universal Pictures,0.132111
fenglish,0.132816
fproduction_Paramount Pictures,0.137068


In [23]:
corrcumlative.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
   .set_table_styles(magnify())

,fcumlative
fproduction_other,-0.3
fdrama,-0.23
certificate_R,-0.15
certificate_Unrated,-0.14
fimd,0.11
fcinematographers,0.11
fmulti_lang,0.12
fproduction_Universal Pictures,0.13
fenglish,0.13
fproduction_Paramount Pictures,0.14


In [19]:
corrcumlative = corr[['fimdb_rating']]
corrcumlative.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
   .set_table_styles(magnify())

,fimdb_rating
fruntime,0.34
fcumlative,0.19
fimdb_rating,1
fimdb_votes,0.35
fimd,0.6
fdirector,0.17
fwriter,0.13
fassistant_directors,0.026
fcinematographers,0.088
fcomposers,0.12


In [20]:
corrcumlative = corr[[ 'Won']].sort_values('Won')
corrcumlative.style.background_gradient(cmap, axis=1)\
    .set_properties(**{'max-width': '80px', 'font-size': '10pt'})\
    .set_caption("Hover to magify")\
    .set_precision(2)\
   .set_table_styles(magnify())

,Won
certificate_Unrated,-0.065
fproduction_other,-0.06
fcomedy,-0.055
fhorror,-0.053
faction,-0.029
fcrime,-0.027
fthriller,-0.02
fGross_y,-0.018
fsport,-0.016
fproduction_Lionsgate,-0.015


In [21]:
# plt.figure(figsize=(6,6))
# sns.heatmap(df[[ 'fcomedy', 'fadult', 'fdocumentary', 'fmystery',
#        'fadventure', 'fbiography', 'fshort', 'fnews', 'fthriller', 'ffamily',
#        'fhistory', 'fcrime', 'fhorror', 'ftalk-show', 'fwar', 'fgame-show',
#        'freality-tv', 'fwestern', 'faction', 'fmusical', 'fsport', 'fmusic',
#        'fanimation', 'fromance', 'fsci-fi', 'ffantasy', 'fdrama']].sort_values(by=['fcumlative_y'],ascending=False),
#             vmin=-1,
#             cmap='coolwarm',
#             annot=True);